# What's this?

This is all the code that was necessary to transform the accepted loan offers, rejected loan offers, and transactional data into the final data format.

On a high level, we need to iterate through each row in the merchant data, and for each business in that row, we check the date of the application.
Then, we try to find the preceding 90 days' worth of transactions for that business, and calculate the total volume of transactions processed, and total successful and failed transactions.
We then print this "collated" form of the data to a new row in a new CSV file.

This shouldn't be necessary to do twice.

TODO: State the file name of the completed output file.

## Accepted transactions

In [ ]:
import pandas
from datetime import datetime, timedelta

accepted = pandas.read_csv('preliminary_data/accepted_merchants.csv', index_col='buuid')
accepted_txn = pandas.read_csv('preliminary_data/accepted_txn.csv', index_col='buuid')

output_list = []

for business in accepted.itertuples():
    # The date that the plan was created is at index 8
    app_date = business[8]
    buuid = business[0]
    
    # Filter the accepted transactions for this business, 
    # and only consider the ones that are within 90 days before
    # the application date
    
    app_date_datetime = datetime.strptime(app_date.split(" ")[0], '%Y-%m-%d')
    app_date_minus_ninety = app_date_datetime - timedelta(days=90)

    app_date_str = app_date_datetime.date().strftime('%Y-%m-%d')
    app_date_minus_ninety_str = app_date_minus_ninety.date().strftime('%Y-%m-%d')
    
    bus_txns = accepted_txn.loc[[buuid]]
    filtered_txns = bus_txns[(bus_txns['transaction_date'] < app_date_str) & (bus_txns['transaction_date'] > app_date_minus_ninety_str)]
    
    # Now we have filtered the transactions we need to calculate the total transaction volume
    # As well as total amount of accepted and declined transactions

    business_total_volume = filtered_txns['transaction_value'].sum()
    business_total_accepted_txns = filtered_txns['approved_transaction_count'].sum()
    business_total_failed_txns = filtered_txns['failed_transaction_count'].sum()
    
    output_list.append([buuid, business_total_volume, business_total_accepted_txns, business_total_failed_txns, "Y"])

print('Writing the following table to CSV.')
output_df = pandas.DataFrame(output_list, columns=["buuid", "txn_vol", "accepted_txn", "failed_txn", "capital_granted"])
print(output_df.head(10))
output_df.to_csv('preliminary_data/accepted_output.csv', index=False)

## Rejected transactions

In [ ]:
import pandas
from datetime import datetime, timedelta

# See the above accepted block for more thorough code comments.

failed = pandas.read_csv('preliminary_data/rejected_merchants_old.csv', index_col='buuid')
failed_txn = pandas.read_csv('preliminary_data/rejected_txn_old.csv', index_col='buuid')

output_list = []

for business in failed.itertuples():
    app_date = business[3]
    buuid = business.Index
    
    app_date_datetime = datetime.strptime(app_date.split(" ")[0], '%Y-%m-%d')
    app_date_minus_ninety = app_date_datetime - timedelta(days=90)

    app_date_str = app_date_datetime.date().strftime('%Y-%m-%d')
    app_date_minus_ninety_str = app_date_minus_ninety.date().strftime('%Y-%m-%d')
    
    try:
        bus_txns = failed_txn.loc[[buuid]]    
        filtered_txns = bus_txns[(bus_txns['transaction_date'] < app_date_str) & (bus_txns['transaction_date'] > app_date_minus_ninety_str)]
    
        business_total_volume = filtered_txns['transaction_value'].sum()
        business_total_accepted_txns = filtered_txns['approved_transaction_count'].sum()
        business_total_failed_txns = filtered_txns['failed_transaction_count'].sum()
    except KeyError:
        business_total_volume = 0
        business_total_accepted_txns = 0
        business_total_failed_txns = 0
        
    output_list.append([buuid, business_total_volume, business_total_accepted_txns, business_total_failed_txns, "N"])

print('Writing the following table to CSV.')
output_df = pandas.DataFrame(output_list, columns=["buuid", "txn_vol", "accepted_txn", "failed_txn", "capital_granted"])
print(output_df.head(10))
output_df.to_csv('preliminary_data/rejected_output.csv', index=False)

## Zip up the two output files, and randomize their order

In [23]:
import pandas

accepted = pandas.read_csv('preliminary_data/rejected_output.csv', index_col='buuid')

rejected = pandas.read_csv('preliminary_data/accepted_output.csv', index_col='buuid')

collated = pandas.concat([accepted, rejected])

collated.sample(frac=1).to_csv('output_data/collated_output.csv', index=False)

print("Verily, your output is ready, m'lord.")

Verily, your output is ready, m'lord.
